# GA4 BigQuery Data Profiling & Flattening

Profiling and denormalization of GA4 exported data from `bigquery-public-data.ga4_obfuscated_sample_ecommerce` (Google Merchandise Store, Nov 2020 – Jan 2021).


## 1. Setup & Connection

In [31]:
from google.colab import auth
from google.cloud import bigquery
import pandas as pd

auth.authenticate_user()

PROJECT_ID = "ga4sb-487512" # replace with your GCP project ID
DATASET = "bigquery-public-data.ga4_obfuscated_sample_ecommerce"
DEST_DATASET = "ga4_flat"  # replace with your destination dataset

In [32]:
client = bigquery.Client(project=PROJECT_ID)

def run_query(sql: str) -> pd.DataFrame:
    """Execute BQ query and return DataFrame."""
    return client.query(sql).to_dataframe()

# verify connection
run_query(f"""
    SELECT COUNT(*) AS table_count
    FROM `{DATASET}.INFORMATION_SCHEMA.TABLES`
""")

,table_count
0,92


**Result:** Connected to project. Dataset contains 92 tables.

## 2. Dataset Overview

Date range and number of tables:

In [33]:
sql = r"""
    SELECT
        MIN(PARSE_DATE('%Y%m%d', REGEXP_EXTRACT(table_name, r'events_(\d+)'))) AS date_from,
        MAX(PARSE_DATE('%Y%m%d', REGEXP_EXTRACT(table_name, r'events_(\d+)'))) AS date_to,
        COUNT(*) AS tables
    FROM `{dataset}`.INFORMATION_SCHEMA.TABLES
    WHERE table_name LIKE 'events_%'
""".format(dataset=DATASET)

run_query(sql)

,date_from,date_to,tables
0,2020-11-01,2021-01-31,92


Total events across all tables:

In [34]:
run_query(f"""
    SELECT COUNT(*) AS total_events
    FROM `{DATASET}.events_*`
""")

,total_events
0,4295584


Event distribution by event_name:

In [35]:
run_query(f"""
    SELECT event_name, COUNT(*) AS events
    FROM `{DATASET}.events_*`
    GROUP BY 1
    ORDER BY 2 DESC
""")

,event_name,events
0,page_view,1350428
1,user_engagement,1058721
2,scroll,493072
3,view_item,386068
4,session_start,354970
5,first_visit,257462
6,view_promotion,190104
7,add_to_cart,58543
8,begin_checkout,38757
9,select_item,31007


> **Result:** 92 daily tables covering 2020-11-01 to 2021-01-31. Total 4,295,584 events across 17 event types. Top events: page_view, user_engagement, scroll.

## 3. Schema Inspection

Identify nested (REPEATED RECORD) vs flat fields:

In [36]:
table_ref = client.get_table(f"{DATASET}.events_20210131")

for field in table_ref.schema:
    if field.field_type == "RECORD":
        print(f"📦 {field.name} ({field.mode} {field.field_type})")
        for sub in field.fields:
            if sub.field_type == "RECORD":
                print(f"    📦 {sub.name} ({sub.mode} {sub.field_type})")
                for subsub in sub.fields:
                    print(f"        - {subsub.name}: {subsub.field_type}")
            else:
                print(f"    - {sub.name}: {sub.field_type}")
    else:
        print(f"  {field.name}: {field.field_type} ({field.mode})")

  event_date: STRING (NULLABLE)
  event_timestamp: INTEGER (NULLABLE)
  event_name: STRING (NULLABLE)
📦 event_params (REPEATED RECORD)
    - key: STRING
    📦 value (NULLABLE RECORD)
        - string_value: STRING
        - int_value: INTEGER
        - float_value: FLOAT
        - double_value: FLOAT
  event_previous_timestamp: INTEGER (NULLABLE)
  event_value_in_usd: FLOAT (NULLABLE)
  event_bundle_sequence_id: INTEGER (NULLABLE)
  event_server_timestamp_offset: INTEGER (NULLABLE)
  user_id: STRING (NULLABLE)
  user_pseudo_id: STRING (NULLABLE)
📦 privacy_info (NULLABLE RECORD)
    - analytics_storage: INTEGER
    - ads_storage: INTEGER
    - uses_transient_token: STRING
📦 user_properties (REPEATED RECORD)
    - key: INTEGER
    📦 value (NULLABLE RECORD)
        - string_value: INTEGER
        - int_value: INTEGER
        - float_value: INTEGER
        - double_value: INTEGER
        - set_timestamp_micros: INTEGER
  user_first_touch_timestamp: INTEGER (NULLABLE)
📦 user_ltv (NULLABLE R

Nested fields requiring UNNEST for flattening:
- `event_params` — REPEATED RECORD (key-value pairs)
- `user_properties` — REPEATED RECORD (key-value pairs)
- `items` — REPEATED RECORD (product-level data)

All other RECORD fields (device, geo, traffic_source, ecommerce, privacy_info) are NULLABLE — accessible via dot notation without UNNEST.

## 4. Profiling: event_params

### 4.1 Unique keys (single day sample)

In [37]:
run_query(f"""
    SELECT DISTINCT ep.key
    FROM `{DATASET}.events_20210131`,
    UNNEST(event_params) AS ep
    ORDER BY 1
""")

,key
0,all_data
1,campaign
2,clean_event
3,coupon
4,currency
5,dclid
6,debug_mode
7,engaged_session_event
8,engagement_time_msec
9,entrances


**Result:** 28 unique event_params keys found in single day sample.



### 4.2 Value types and fill rates per key (single day)



In [38]:
run_query(f"""
    SELECT
        ep.key,
        COUNT(*) AS total,
        COUNTIF(ep.value.string_value IS NOT NULL) AS string_filled,
        COUNTIF(ep.value.int_value IS NOT NULL) AS int_filled,
        COUNTIF(ep.value.float_value IS NOT NULL) AS float_filled,
        COUNTIF(ep.value.double_value IS NOT NULL) AS double_filled,
        COUNTIF(
            ep.value.string_value IS NULL
            AND ep.value.int_value IS NULL
            AND ep.value.float_value IS NULL
            AND ep.value.double_value IS NULL
        ) AS all_null
    FROM `{DATASET}.events_20210131`,
    UNNEST(event_params) AS ep
    GROUP BY ep.key
    ORDER BY ep.key
""")

,key,total,string_filled,int_filled,float_filled,double_filled,all_null
0,all_data,4804,0,0,0,0,4804
1,campaign,6436,6436,0,0,0,0
2,clean_event,4804,4804,0,0,0,0
3,coupon,12,12,0,0,0,0
4,currency,172,172,0,0,0,0
5,dclid,124,0,0,0,0,124
6,debug_mode,21602,0,21602,0,0,0
7,engaged_session_event,25404,0,25404,0,0,0
8,engagement_time_msec,15499,0,15499,0,0,0
9,entrances,2624,0,2624,0,0,0


**Result:** 4 keys have 100% NULL values: `all_data`, `dclid`, `gclid`, `gclsrc`. `session_engaged` has mixed types (string + int). Remaining keys use either string or int storage consistently.

### 4.3 Full scan — confirm 100% NULL keys across all 92 tables

In [39]:
run_query(f"""
    SELECT ep.key, COUNT(*) AS total,
        COUNTIF(
            ep.value.string_value IS NULL
            AND ep.value.int_value IS NULL
            AND ep.value.float_value IS NULL
            AND ep.value.double_value IS NULL
        ) AS all_null
    FROM `{DATASET}.events_*`,
    UNNEST(event_params) AS ep
    WHERE ep.key IN ('all_data', 'dclid', 'gclid', 'gclsrc')
    GROUP BY ep.key
""")

,key,total,all_null
0,all_data,2828174,2828174
1,dclid,15701,15701
2,gclid,118481,118481
3,gclsrc,116846,116846


**Result:** Confirmed across all 92 tables — `total` = `all_null` for all 4 keys. Safe to exclude.

### 4.4 Mixed type check: session_engaged

In [40]:
run_query(f"""
    SELECT
        COUNTIF(ep.value.string_value IS NOT NULL) AS string_filled,
        COUNTIF(ep.value.int_value IS NOT NULL) AS int_filled,
        COUNT(*) AS total
    FROM `{DATASET}.events_20210131`,
    UNNEST(event_params) AS ep
    WHERE ep.key = 'session_engaged'
""")

,string_filled,int_filled,total
0,21601,2582,24183


**Result:** `session_engaged` — 21,601 string values + 2,582 int values out of 24,183 total. Mixed type confirmed → use `COALESCE(value.string_value, CAST(value.int_value AS STRING))` at flattening.

### 4.5 event_params profiling summary

- 28 unique keys found
- 4 keys excluded (100% NULL): `all_data`, `dclid`, `gclid`, `gclsrc`
- 1 key with mixed types: `session_engaged` → COALESCE needed
- 28 - 4 = **24 keys retained** for flattening

## 5. Profiling: user_properties

### 5.1 Single day sample

In [41]:
run_query(f"""
    SELECT up.key, COUNT(*) AS total
    FROM `{DATASET}.events_20210131`,
    UNNEST(user_properties) AS up
    GROUP BY up.key
""")

,key,total


**Result:** Empty — no user_properties found in single day sample.

### 5.2 Full scan

In [42]:
run_query(f"""
    SELECT COUNT(*) AS total_kv_pairs
    FROM `{DATASET}.events_*`,
    UNNEST(user_properties) AS up
""")

,total_kv_pairs
0,136


**Result:** Single day — empty. Full scan — 136 key-value pairs across all 4.3M events. Excluded entirely from flattening.

## 6. Profiling: items

### 6.1 Items fields and fill rates (single day)

In [43]:
run_query(f"""
    SELECT
        COUNT(*) AS total_items,
        COUNTIF(item_id IS NOT NULL) AS item_id_filled,
        COUNTIF(item_name IS NOT NULL) AS item_name_filled,
        COUNTIF(item_brand IS NOT NULL) AS item_brand_filled,
        COUNTIF(item_variant IS NOT NULL) AS item_variant_filled,
        COUNTIF(item_category IS NOT NULL) AS item_category_filled,
        COUNTIF(item_category2 IS NOT NULL) AS item_category2_filled,
        COUNTIF(item_category3 IS NOT NULL) AS item_category3_filled,
        COUNTIF(item_category4 IS NOT NULL) AS item_category4_filled,
        COUNTIF(item_category5 IS NOT NULL) AS item_category5_filled,
        COUNTIF(price IS NOT NULL) AS price_filled,
        COUNTIF(quantity IS NOT NULL) AS quantity_filled,
        COUNTIF(item_revenue IS NOT NULL) AS item_revenue_filled,
        COUNTIF(coupon IS NOT NULL) AS coupon_filled,
        COUNTIF(affiliation IS NOT NULL) AS affiliation_filled,
        COUNTIF(location_id IS NOT NULL) AS location_id_filled,
        COUNTIF(item_list_name IS NOT NULL) AS item_list_name_filled,
        COUNTIF(item_list_index IS NOT NULL) AS item_list_index_filled,
        COUNTIF(promotion_id IS NOT NULL) AS promotion_id_filled,
        COUNTIF(promotion_name IS NOT NULL) AS promotion_name_filled,
        COUNTIF(creative_name IS NOT NULL) AS creative_name_filled,
        COUNTIF(creative_slot IS NOT NULL) AS creative_slot_filled
    FROM `{DATASET}.events_20210131`,
    UNNEST(items)
""")

,total_items,item_id_filled,item_name_filled,item_brand_filled,item_variant_filled,item_category_filled,item_category2_filled,item_category3_filled,item_category4_filled,item_category5_filled,...,item_revenue_filled,coupon_filled,affiliation_filled,location_id_filled,item_list_name_filled,item_list_index_filled,promotion_id_filled,promotion_name_filled,creative_name_filled,creative_slot_filled
0,19867,19867,19867,19867,19867,19867,19867,19867,19867,19867,...,0,19867,19867,19867,19867,19867,19867,19867,19867,19867


**Result:** 19,867 item rows. Most fields 100% filled. `item_revenue` = 0 on single day. `price` and `quantity` hidden by truncation — checked in 6.3.

### 6.2 Items per event by event_name (single day)

In [44]:
run_query(f"""
    SELECT
        event_name,
        COUNT(*) AS events_with_items,
        SUM(ARRAY_LENGTH(items)) AS total_items,
        ROUND(AVG(ARRAY_LENGTH(items)), 1) AS avg_items_per_event
    FROM `{DATASET}.events_20210131`
    WHERE ARRAY_LENGTH(items) > 0
    GROUP BY 1
    ORDER BY 2 DESC
""")

,event_name,events_with_items,total_items,avg_items_per_event
0,view_item,1221,13186,10.8
1,view_promotion,889,889,1.0
2,add_to_cart,295,3138,10.6
3,select_item,237,2313,9.8
4,begin_checkout,234,258,1.1
5,select_promotion,64,64,1.0
6,purchase,19,19,1.0


**Result:** Data quality issue — `view_item`, `add_to_cart`, `select_item` carry ~10 items per event (should be 1). `purchase`, `begin_checkout`, `view_promotion`, `select_promotion` — 1.0-1.1 avg, as expected. Items not included in flattening — not critical for experiment.

### 6.3 Fill rates across full dataset (quantity, item_revenue)

In [45]:
run_query(f"""
    SELECT
        COUNTIF(quantity IS NOT NULL) AS quantity_filled,
        COUNTIF(item_revenue IS NOT NULL) AS item_revenue_filled,
        COUNT(*) AS total
    FROM `{DATASET}.events_*`,
    UNNEST(items)
""")

,quantity_filled,item_revenue_filled,total
0,150304,15555,3982732


**Result:** Full dataset (3.98M item rows): `quantity` filled 150,304 (3.8%), `item_revenue` filled 15,555 (0.4%). Both populated only on purchase events.

### 6.4 items profiling summary

- Items array present in 7 event types
- Data quality issue: `view_item`, `add_to_cart`, `select_item` have inflated item counts (~10 per event)
- `quantity` and `item_revenue` sparsely filled (purchase events only)
- **Decision: items excluded from flattening** — not critical for session-level experiment

## 7. Profiling: flat fields

### 7.1 NULL check — single day sample

In [46]:
run_query(f"""
    SELECT
        COUNT(*) AS total,
        -- identifiers
        COUNTIF(event_previous_timestamp IS NOT NULL) AS event_prev_ts,
        COUNTIF(event_server_timestamp_offset IS NOT NULL) AS event_server_ts_offset,
        COUNTIF(user_id IS NOT NULL) AS user_id,
        COUNTIF(event_value_in_usd IS NOT NULL) AS event_value_usd,
        -- privacy
        COUNTIF(privacy_info.analytics_storage IS NOT NULL) AS privacy_analytics,
        COUNTIF(privacy_info.ads_storage IS NOT NULL) AS privacy_ads,
        COUNTIF(privacy_info.uses_transient_token IS NOT NULL) AS privacy_transient,
        -- device
        COUNTIF(device.mobile_os_hardware_model IS NOT NULL) AS device_hw_model,
        COUNTIF(device.vendor_id IS NOT NULL) AS device_vendor_id,
        COUNTIF(device.advertising_id IS NOT NULL) AS device_ad_id,
        -- ecommerce
        COUNTIF(ecommerce.refund_value_in_usd IS NOT NULL) AS ecom_refund_usd,
        COUNTIF(ecommerce.refund_value IS NOT NULL) AS ecom_refund,
        COUNTIF(ecommerce.shipping_value_in_usd IS NOT NULL) AS ecom_ship_usd,
        COUNTIF(ecommerce.shipping_value IS NOT NULL) AS ecom_ship
    FROM `{DATASET}.events_20210131`
""")

,total,event_prev_ts,event_server_ts_offset,user_id,event_value_usd,privacy_analytics,privacy_ads,privacy_transient,device_hw_model,device_vendor_id,device_ad_id,ecom_refund_usd,ecom_refund,ecom_ship_usd,ecom_ship
0,26489,0,0,0,0,0,0,26489,0,0,0,0,0,0,0


**Result:** 12 fields are 100% NULL on single day. `event_value_in_usd` — 0 on single day but needs full scan confirmation. `privacy_info.uses_transient_token` — 100% filled.

### 7.2 Full scan — confirm 100% NULL fields across all tables

In [47]:
run_query(f"""
    SELECT
        COUNT(*) AS total,
        COUNTIF(event_previous_timestamp IS NOT NULL) AS event_prev_ts,
        COUNTIF(event_server_timestamp_offset IS NOT NULL) AS event_server_ts_offset,
        COUNTIF(user_id IS NOT NULL) AS user_id,
        COUNTIF(event_value_in_usd IS NOT NULL) AS event_value_usd,
        COUNTIF(privacy_info.analytics_storage IS NOT NULL) AS privacy_analytics,
        COUNTIF(privacy_info.ads_storage IS NOT NULL) AS privacy_ads,
        COUNTIF(device.mobile_os_hardware_model IS NOT NULL) AS device_hw_model,
        COUNTIF(device.vendor_id IS NOT NULL) AS device_vendor_id,
        COUNTIF(device.advertising_id IS NOT NULL) AS device_ad_id,
        COUNTIF(ecommerce.refund_value_in_usd IS NOT NULL) AS ecom_refund_usd,
        COUNTIF(ecommerce.refund_value IS NOT NULL) AS ecom_refund,
        COUNTIF(ecommerce.shipping_value_in_usd IS NOT NULL) AS ecom_ship_usd,
        COUNTIF(ecommerce.shipping_value IS NOT NULL) AS ecom_ship
    FROM `{DATASET}.events_*`
""")

,total,event_prev_ts,event_server_ts_offset,user_id,event_value_usd,privacy_analytics,privacy_ads,device_hw_model,device_vendor_id,device_ad_id,ecom_refund_usd,ecom_refund,ecom_ship_usd,ecom_ship
0,4295584,0,0,0,5242,0,0,0,0,0,0,0,0,0


**Result:** Confirmed across all tables — 12 fields 100% NULL. `event_value_in_usd` has 5,242 filled rows (0.12%) — retained.

### 7.3 flat fields profiling summary

12 fields excluded (100% NULL):
- `event_previous_timestamp`, `event_server_timestamp_offset`, `user_id`
- `privacy_info.analytics_storage`, `privacy_info.ads_storage`
- `device.mobile_os_hardware_model`, `device.vendor_id`, `device.advertising_id`
- `ecommerce.refund_value_in_usd`, `ecommerce.refund_value`, `ecommerce.shipping_value_in_usd`, `ecommerce.shipping_value`

Retained: all other flat fields including `event_value_in_usd` (sparse but present).

## 8. Profiling Summary

### Excluded from flattening:

**event_params (4 keys, 100% NULL):** `all_data`, `dclid`, `gclid`, `gclsrc`

**user_properties:** entirely (136 key-value pairs out of 4.3M events)

**items:** entirely (data quality issues — inflated item counts on view_item, add_to_cart, select_item; not critical for session-level experiment)

**flat fields (12, 100% NULL):**
- `event_previous_timestamp`, `event_server_timestamp_offset`, `user_id`
- `privacy_info.analytics_storage`, `privacy_info.ads_storage`
- `device.mobile_os_hardware_model`, `device.vendor_id`, `device.advertising_id`
- `ecommerce.refund_value_in_usd`, `ecommerce.refund_value`, `ecommerce.shipping_value_in_usd`, `ecommerce.shipping_value`

### Retained:

- **24 event_params keys** (1 with mixed type: `session_engaged` → COALESCE)
- **All remaining flat fields** (device, geo, traffic_source, ecommerce, privacy_info.uses_transient_token, event_value_in_usd)

### Known issues:

- `session_engaged` — mixed string/int storage → handled with COALESCE
- `collected_traffic_source` — does not exist in this dataset (added to GA4 BQ export after 2021)

## 9. Flattening Query

Build and execute the flattening query — pivot 24 event_params keys into columns, keep all retained flat fields. Save result to `ga4_flat.flatten_no_item` table.

In [48]:
DEST_TABLE = f"{PROJECT_ID}.{DEST_DATASET}.flatten_no_item_colab"

sql = f"""
    SELECT
        -- identifiers
        event_date,
        event_name,
        event_timestamp,
        event_bundle_sequence_id,
        event_value_in_usd,
        user_pseudo_id,
        user_first_touch_timestamp,
        stream_id,
        platform,
        -- privacy
        privacy_info.uses_transient_token,
        -- device
        device.category AS device_category,
        device.mobile_brand_name AS device_mobile_brand,
        device.mobile_model_name AS device_mobile_model,
        device.mobile_marketing_name AS device_mobile_marketing,
        device.operating_system AS device_os,
        device.operating_system_version AS device_os_version,
        device.language AS device_language,
        device.is_limited_ad_tracking AS device_limited_ad_tracking,
        device.web_info.browser AS browser,
        device.web_info.browser_version AS browser_version,
        -- geo
        geo.continent,
        geo.country,
        geo.region,
        geo.city,
        geo.sub_continent,
        geo.metro,
        -- traffic_source (first touch)
        traffic_source.source AS traffic_source,
        traffic_source.medium AS traffic_medium,
        traffic_source.name AS traffic_campaign,
        -- ecommerce
        ecommerce.total_item_quantity AS ecom_total_item_qty,
        ecommerce.purchase_revenue_in_usd AS ecom_revenue_usd,
        ecommerce.purchase_revenue AS ecom_revenue,
        ecommerce.tax_value_in_usd AS ecom_tax_usd,
        ecommerce.tax_value AS ecom_tax,
        ecommerce.unique_items AS ecom_unique_items,
        ecommerce.transaction_id AS ecom_transaction_id,
        -- event_params (24 keys)
        (SELECT MAX(value.string_value) FROM UNNEST(event_params) WHERE key = 'page_location') AS page_location,
        (SELECT MAX(value.string_value) FROM UNNEST(event_params) WHERE key = 'page_title') AS page_title,
        (SELECT MAX(value.string_value) FROM UNNEST(event_params) WHERE key = 'page_referrer') AS page_referrer,
        (SELECT MAX(value.string_value) FROM UNNEST(event_params) WHERE key = 'source') AS ep_source,
        (SELECT MAX(value.string_value) FROM UNNEST(event_params) WHERE key = 'medium') AS ep_medium,
        (SELECT MAX(value.string_value) FROM UNNEST(event_params) WHERE key = 'campaign') AS ep_campaign,
        (SELECT MAX(value.string_value) FROM UNNEST(event_params) WHERE key = 'term') AS ep_term,
        (SELECT MAX(value.int_value) FROM UNNEST(event_params) WHERE key = 'ga_session_id') AS ga_session_id,
        (SELECT MAX(value.int_value) FROM UNNEST(event_params) WHERE key = 'ga_session_number') AS ga_session_number,
        (SELECT MAX(value.int_value) FROM UNNEST(event_params) WHERE key = 'engagement_time_msec') AS engagement_time_msec,
        (SELECT MAX(value.int_value) FROM UNNEST(event_params) WHERE key = 'entrances') AS entrances,
        (SELECT MAX(value.int_value) FROM UNNEST(event_params) WHERE key = 'debug_mode') AS debug_mode,
        (SELECT MAX(value.int_value) FROM UNNEST(event_params) WHERE key = 'engaged_session_event') AS engaged_session_event,
        (SELECT MAX(COALESCE(value.string_value, CAST(value.int_value AS STRING))) FROM UNNEST(event_params) WHERE key = 'session_engaged') AS session_engaged,
        (SELECT MAX(value.string_value) FROM UNNEST(event_params) WHERE key = 'clean_event') AS clean_event,
        (SELECT MAX(value.string_value) FROM UNNEST(event_params) WHERE key = 'link_domain') AS link_domain,
        (SELECT MAX(value.string_value) FROM UNNEST(event_params) WHERE key = 'link_url') AS link_url,
        (SELECT MAX(value.string_value) FROM UNNEST(event_params) WHERE key = 'outbound') AS outbound,
        (SELECT MAX(value.int_value) FROM UNNEST(event_params) WHERE key = 'percent_scrolled') AS percent_scrolled,
        (SELECT MAX(value.string_value) FROM UNNEST(event_params) WHERE key = 'search_term') AS search_term,
        (SELECT MAX(value.int_value) FROM UNNEST(event_params) WHERE key = 'unique_search_term') AS unique_search_term,
        (SELECT MAX(value.string_value) FROM UNNEST(event_params) WHERE key = 'promotion_name') AS ep_promotion_name,
        (SELECT MAX(value.string_value) FROM UNNEST(event_params) WHERE key = 'coupon') AS ep_coupon,
        (SELECT MAX(value.string_value) FROM UNNEST(event_params) WHERE key = 'currency') AS ep_currency
    FROM `{DATASET}.events_*`
"""

job_config = bigquery.QueryJobConfig(
    destination=DEST_TABLE,
    write_disposition="WRITE_TRUNCATE"
)

job = client.query(sql, job_config=job_config)
job.result()

table = client.get_table(DEST_TABLE)
print(f"Loaded {table.num_rows} rows to {DEST_TABLE}")

Loaded 4295584 rows to ga4sb-487512.ga4_flat.flatten_no_item_colab


Preview — first and last 10 rows:

In [49]:
run_query(f"""
    (SELECT * FROM `{DEST_TABLE}` ORDER BY event_timestamp ASC LIMIT 10)
    UNION ALL
    (SELECT * FROM `{DEST_TABLE}` ORDER BY event_timestamp DESC LIMIT 10)
""")

,event_date,event_name,event_timestamp,event_bundle_sequence_id,event_value_in_usd,user_pseudo_id,user_first_touch_timestamp,stream_id,platform,uses_transient_token,...,clean_event,link_domain,link_url,outbound,percent_scrolled,search_term,unique_search_term,ep_promotion_name,ep_coupon,ep_currency
0,20210131,view_promotion,1612137595412363,-7832409636,NaN,38749961.2390394784,1612137591044846,2100450278,WEB,No,...,None,None,None,None,<NA>,None,<NA>,None,None,None
1,20210131,page_view,1612137595412363,-7832409636,NaN,38749961.2390394784,1612137591044846,2100450278,WEB,No,...,None,None,None,None,<NA>,None,<NA>,None,None,None
2,20210131,user_engagement,1612137595412363,-7832409636,NaN,38749961.2390394784,1612137591044846,2100450278,WEB,No,...,None,None,None,None,<NA>,None,<NA>,None,None,None
3,20210131,page_view,1612137594824131,1797586051,NaN,8728288.1164272892,1612136955500305,2100450278,WEB,No,...,None,None,None,None,<NA>,None,<NA>,None,None,None
4,20210131,view_promotion,1612137594824131,1797586051,NaN,8728288.1164272892,1612136955500305,2100450278,WEB,No,...,None,None,None,None,<NA>,None,<NA>,None,None,None
5,20210131,first_visit,1612137591044846,8372346140,NaN,38749961.2390394784,1612137591044846,2100450278,WEB,No,...,None,None,None,None,<NA>,None,<NA>,None,None,None
6,20210131,session_start,1612137591044846,8372346140,NaN,38749961.2390394784,1612137591044846,2100450278,WEB,No,...,None,None,None,None,<NA>,None,<NA>,None,None,None
7,20210131,page_view,1612137591044846,8372346140,NaN,38749961.2390394784,1612137591044846,2100450278,WEB,No,...,gtm.js,None,None,None,<NA>,None,<NA>,None,None,None
8,20210131,session_start,1612137580397192,-5370042247,NaN,1299518.4337551586,1612137580397192,2100450278,WEB,No,...,None,None,None,None,<NA>,None,<NA>,None,None,None
9,20210131,page_view,1612137580397192,-5370042247,NaN,1299518.4337551586,1612137580397192,2100450278,WEB,No,...,gtm.js,None,None,None,<NA>,None,<NA>,None,None,None


**Result:** 20 rows × 60 columns. First rows from 2021-01-31, last rows from 2020-11-01. All flat fields and pivoted event_params present. Table is flat — ready for use.

## 10. Flat Table Validation

### 10.1 Row count match

In [50]:
run_query(f"""
    SELECT
        (SELECT COUNT(*) FROM `{DATASET}.events_*`) AS source_events,
        (SELECT COUNT(*) FROM `{DEST_TABLE}`) AS flat_events
""")

,source_events,flat_events
0,4295584,4295584


**Result:** Row count match — 4,295,584 = 4,295,584. No rows lost or duplicated.

### 10.2 Date range match

In [51]:
run_query(f"""
    SELECT
        MIN(event_date) AS date_from,
        MAX(event_date) AS date_to,
        COUNT(DISTINCT event_date) AS unique_dates
    FROM `{DEST_TABLE}`
""")

,date_from,date_to,unique_dates
0,20201101,20210131,92


**Result:** Date range match — 2020-11-01 to 2021-01-31, 92 unique dates.

### 10.3 Event distribution match

In [52]:
run_query(f"""
    SELECT event_name, COUNT(*) AS events
    FROM `{DEST_TABLE}`
    GROUP BY 1
    ORDER BY 2 DESC
""")

,event_name,events
0,page_view,1350428
1,user_engagement,1058721
2,scroll,493072
3,view_item,386068
4,session_start,354970
5,first_visit,257462
6,view_promotion,190104
7,add_to_cart,58543
8,begin_checkout,38757
9,select_item,31007


**Result:** 17 event types (including `view_item_list` which was absent in single day sample). Distribution matches source. Validation passed — flat table is complete and consistent.

### 10.4 Validation summary

All checks passed:
- Row count: source 4,295,584 = flat 4,295,584
- Date range: 2020-11-01 to 2021-01-31, 92 days
- Event types: 17 (matches source)
- Columns: 60 (flat, no nested structures)

Flat table `ga4_flat.flatten_no_item` is ready for downstream use.